In [1]:
import numpy as np
import math
import warnings
import math
import re


import tables as tt
import stat_funcs as sf

warnings.filterwarnings('ignore')


In [2]:




class Apply_Zombie_Template:
    def __init__(self,data):
        self.data = data
        
        self.zomb_nat_ac = {
                    'Fine':0,
                    'Diminutive':0,
                    'Tiny':0,
                    'Small':1,
                    'Medium':2,
                    'Large':3,
                    'Huge':4,
                    'Gargantuan':7,
                    'Colossal':11,
                            }

        self.zomb_slam_dam = {
                    'Fine':'1',
                    'Diminutive':'1d2',
                    'Tiny':'1d3',
                    'Small':'1d4',
                    'Medium':'1d6',
                    'Large':'1d8',
                    'Huge':'2d6',
                    'Gargantuan':'2d8',
                    'Colossal':'4d6',
                                }
        self.zomb_CR_table = {
                    '1/2': '0.17',
                    '0.5': '0.17',
                    '1':'0.25',
                    '2':'0.5',
                    '3':'1',
                    '4':'1',
                    '5':'2',
                    '6':'2',
                    '7':'3',
                    '8':'3',
                    '9':'4',
                    '10':'4',
                    '11':'5',
                    '12':'5',
                    '13':'6',
                    '14':'6',
                    '15':'6',
                    '16':'6',
                    '17':'7',
                    '18':'7',
                    '19':'7',
                    '20':'7',
                    '21':'8',
                    '22':'8',
                    '23':'8',
                    '24':'8',
                    '25':'9',
                    '26':'9',
                    '27':'9',
                    '28':'9',
                            }
        self.zomb_bonus_HD = {
                    'Fine':0,
                    'Diminutive':0,
                    'Tiny':0,
                    'Small':1,
                    'Medium':1,
                    'Large':2,
                    'Huge':4,
                    'Gargantuan':6,
                    'Colossal':10,
        }
        self.subtypes_to_remove = ['good', 'evil', 'lawful', 'chaotic', 'giant', 'dwarf', 'human', 'elf', 
                   'gnome', 'orc', 'catfolk', 'goblin','goblinoid','halfling','kitsune', 'ratfolk']
        
    def _zombie_HD(self, total_hit_dice):
        class_list = list(filter(None, re.split('(\d)|(\/)',str(self.data['Class']))))
        levels = [int(i) for i in class_list if i.isnumeric()]
        for lvl in levels:
            total_hit_dice -= lvl
        if total_hit_dice > 0 and total_hit_dice < 1.0:
            total_hit_dice = str(0.5)
        else:
            total_hit_dice = int(max(1, total_hit_dice))
        total_hit_dice += self.zomb_bonus_HD[self.data['Size']]
        return total_hit_dice
    
    def _zombie_AC(self, stats):
        total_AC = 10
        nat = self.zomb_nat_ac[self.data['Size']]
        dex = math.floor((float(stats[1][1])-10)/2)
        size = int(tt.size_bonus[self.data['Size']])
        total_AC += nat
        total_AC += dex
        total_AC += size
        ff = 10 + nat + size
        if dex < 0:
            ff += dex
        touch = 10 + dex + size
        mods = '('
        if dex > 0:
            mods += '+' + str(dex) + ' Dex'
        elif dex < 0:
            mods += str(dex) + ' Dex'
        if nat > 0:
            mods += ', +' + str(nat) + ' natural'
        elif nat < 0:
            mods += ', ' + str(nat) + ' natural'
        if size > 0:
            mods += ', +' + str(size) + ' size'
        elif size < 0:
            mods += ', ' + str(size) + ' size'
        mods += ')'
        if dex == 0 and nat ==0:
            mods =''
        AC_all = str(total_AC) + ', touch ' + str(touch) + ', flat-footed ' + str(ff)

        return mods, AC_all

    def _get_zomb_saves(self, total_hit_dice, stats):
        saves = sf.get_saves(self.data)
        saves[0][1] = str(math.floor(total_hit_dice/3) + math.floor((float(stats[5][1])-10)/2))
        saves[1][1] = str(math.floor(total_hit_dice/3) + math.floor((float(stats[1][1])-10)/2))
        saves[2][1] = str(math.floor(total_hit_dice/2)+2 + math.floor((float(stats[4][1])-10)/2))
        saves = ', '.join([' '.join(save) for save in saves])
        return saves

    def _get_zomb_stats(self, stats):
        stats[0][1] = str(int(stats[0][1])+2)
        stats[1][1] = str(int(stats[1][1])-2)
        stats[2][1] = '-'
        stats[3][1] = '-'
        stats[4][1] = '10'
        stats[5][1] = '10'
        stats_str = ', '.join([' '.join(stat) for stat in stats])
        return stats, stats_str
    
    def _get_zomb_ranged(self, old_bab, stats):
        highest_ab = int(self.data['Base Atk'])+math.floor((float(stats[1][1])-10)/2) + int(tt.size_bonus[self.data['Size']])
        if math.floor((float(stats[0][1])-10)/2) >= 0:
            pm = '+'
        else:
            pm =''
        
        ranged = sf.get_attacks(self.data, me_ra='Ranged')
        new_ranged = []
        for fr in range(len(ranged)):
            for at in range(len(ranged[fr])):
                if ranged[fr][at][0][0].isalpha():
                    wep = ranged[fr][at][0]
                else:
                    wep = ranged[fr][at][1]
                iterative_bonus = int(self.data['Base Atk'])
                na = 0
                full_attack_bonus = ''
                while iterative_bonus >= 0:
                    bonus = highest_ab - na*5
                    if bonus >= 0:
                        bonus = '+' + str(bonus)
                    else:
                        if str(bonus)[0] != '-':
                            bonus = '-' + str(bonus)
                        else:
                            bonus = str(bonus)
                    full_attack_bonus += bonus
                    if iterative_bonus - 5>= 0:
                        full_attack_bonus += '/'
                    na += 1
                    iterative_bonus -= 5
                wep_damage = re.findall('(\d*d\d*)[+-]', ranged[fr][at][-1][1:])[0]
                try:
                    crit=tt.weapons_with_crit[wep.lower()]
                except:
                    crit=''
                if 'composite' in wep:
                    extra_damage = math.floor((float(stats[0][1])-10)/2)
                else:
                    extra_damage = 0
                if math.floor(extra_damage) >= 0:
                    extra_damage = '+' + str(extra_damage)
                full_damage = wep_damage+str(extra_damage)
                new_ranged.append([' '.join([wep, full_attack_bonus, '(' + full_damage + ' ' +crit+')'])])
        new_ranged = [', '.join(i) for i in new_ranged]
        new_ranged = ' or '.join(new_ranged)
        return new_ranged
        

    
    def _get_zomb_melee(self, old_bab, stats):
        highest_ab = int(self.data['Base Atk'])+math.floor((float(stats[0][1])-10)/2) + int(tt.size_bonus[self.data['Size']])
        damage = self.zomb_slam_dam[self.data['Size']]
        if math.floor((float(stats[0][1])-10)/2) >= 0:
            pm = '+'
        else:
            pm =''
        if highest_ab >= 0:
            slam_attack = 'slam +' + str(highest_ab) + ' (' + damage + pm + str(math.floor((float(stats[0][1])-10)/2)) + ')'
        else:
            slam_attack = 'slam ' + str(highest_ab) + ' (' + damage + pm + str(math.floor((float(stats[0][1])-10)/2)) + ')'
        
        melee = sf.get_attacks(self.data, me_ra='Melee')
        if melee == [[]]:
            new_melee = slam_attack
        else:
            new_melee = []
            nat_full = []
            for fr in range(len(melee)):
                for at in range(len(melee[fr])):
                    if melee[fr][at][0][0].isalpha():
                        wep = melee[fr][at][0]
                    else:
                        wep = melee[fr][at][1]
                    natural_attack=False
                    for nattack in tt.nat_attack_list:
                        if nattack in wep:
                            natural_attack=True
                            break
                    if not natural_attack:
                    #if wep not in tt.nat_attack_list:
                        iterative_bonus = int(self.data['Base Atk'])
                        na = 0
                        full_attack_bonus = ''
                        while iterative_bonus >= 0:
                            bonus = highest_ab - na*5
                            if bonus >= 0:
                                bonus = '+' + str(bonus)
                            else:
                                if str(bonus)[0] != '-':
                                    bonus = '-' + str(bonus)
                                else:
                                    bonus = str(bonus)
                            full_attack_bonus += bonus
                            if iterative_bonus - 5>= 0:
                                full_attack_bonus += '/'
                            na += 1
                            iterative_bonus -= 5
                        wep_damage = re.findall('(\d*d\d*)[+-]', melee[fr][at][-1][1:])[0]
                        try:
                            crit=tt.weapons_with_crit[wep.lower()]
                        except:
                            crit=''
                        extra_damage = math.floor((float(stats[0][1])-10)/2)
                        if math.floor(extra_damage) >= 0:
                            extra_damage = '+' + str(extra_damage)
                        full_damage = wep_damage+str(extra_damage)
                        new_melee.append([' '.join([wep, full_attack_bonus, '(' + full_damage + ' ' +crit+ ')'])])
                    else:
                        if 'slam' not in wep:
                            if wep[:4] == ' and' or wep[:4]=='and ':
                                wep = wep[4:]
                            if melee[fr][at][0][0].isnumeric():
                                wep = melee[fr][at][0][0] + ' ' + wep
                            bonus = highest_ab
                            
                            pri_sec='Secondary'
                            for key in tt.nat_pri_sec:
                                if key in wep:
                                    pri_sec = tt.nat_pri_sec[key]
                                    break
                            if pri_sec !='Primary':
                                bonus -= 5
                                
                            if bonus >= 0:
                                bonus = '+' + str(bonus)
                            else:
                                if str(bonus)[0] != '-':
                                    bonus = '-' + str(bonus)
                                else:
                                    bonus = str(bonus)
                            plus=''
                            if 'plus' in melee[fr][at][-1][1:]:
                                plus = ' ' + re.findall('(plus\s*.*)\)',melee[fr][at][-1][1:])[0] 
                            if nattack =='bite' or nattack =='gore' or nattack =='pincer' or nattack =='tail slap' or nattack =='':
                                wep_damage = tt.big_nat_damage[self.data['Size']]
                            else:
                                wep_damage = tt.small_nat_damage[self.data['Size']]
                            extra_damage = math.floor((float(stats[0][1])-10)/2)
                            if pri_sec !='Primary':
                                extra_damage = math.floor(extra_damage/2)
                            if math.floor(extra_damage) >= 0:
                                extra_damage = '+' + str(extra_damage)
                            full_damage = wep_damage+str(extra_damage)+plus
                            nat_full.append(' '.join([wep, bonus, '(' + full_damage + ')']))

            nat_full.append(slam_attack)
            new_melee.append(nat_full)
            new_melee = [', '.join(i) for i in new_melee]
            new_melee = ' or '.join(new_melee)
        return new_melee
    
    def zombie_temp(self, replace_fly=True):
        total_hit_dice, hit_dice = sf.get_hit_dice(self.data)
        if float(total_hit_dice) > 28:
            print('Total zombified hit dice is ', total_hit_dice, 'but the template only goes to 28.')
            print('Setting new hit dice to 28.')
            total_hit_dice = 28
            
        if self.data['Type'] == 'undead':
            print('Creature is already undead!!!')
            return 0
        self.data['Name'] += ' Zombie'
        self.data['Type'] = 'undead'
        for subtype in self.subtypes_to_remove:
            self.data['SubType'].replace(subtype,'')
        
        total_hit_dice = self._zombie_HD(total_hit_dice)
        self.data['HD'] = str(total_hit_dice) + 'd8+' + str(max(3,total_hit_dice))
        self.data['HP'] = str(math.floor(float(total_hit_dice) * 4.5)+max(3,total_hit_dice))
        self.data['CR'] = self.zomb_CR_table[str(total_hit_dice)]
        self.data['XP'] = tt.exp_by_CR[self.data['CR']]
        
        if 'fly' in self.data['Speed']:
            if replace_fly:
                print('Changing fly speed maneuverability to clumsy!')
                print('If the base creature flew through magical means, rerun with replace_fly=False')
                fly_speed = re.findall('(fly.*\(?[a-zA-Z]*\);?,?\s*)', self.data['Speed'])[0]
                new_fly_speed = re.findall('(fly.*)\(', fly_speed)[0] + '(clumsy)'
                front_of_speed = re.findall('([\S\s]*)fly.*\(?[a-zA-Z]*\);?,?\s*', self.data['Speed'])[0]
                back_of_speed = re.findall('fly.*\(?[a-zA-Z]*\)(;?,?\s*.*)', self.data['Speed'])[0]
                self.data['Speed'] = front_of_speed + ' ' + new_fly_speed + ' ' + back_of_speed
        if 'weapon training' in self.data['SQ']:
            weapon_training = re.findall('weapon training\s*(\d)')
        self.data['Alignment'] = 'NE'
        stats = sf.get_stats(self.data)
        stats, stats_str = self._get_zomb_stats(stats)
        self.data['Abilitiy Scores'] = stats_str
        old_bab = float(self.data['Base Atk'])
        self.data['Base Atk'] = str(math.floor(int(total_hit_dice)*0.75))
        self.data['Skills'] = ''
        self.data['Senses'] = 'darkvision 60 ft.'
        self.data['Defensive Abilities'] = ''
        self.data['DR'] = '5/Slashing'
        self.data['Immune'] = 'undead traits'
        self.data['Saves'] = self._get_zomb_saves(float(total_hit_dice), stats)
        self.data['Feats'] = 'Toughness'
        self.data['Melee'] = self._get_zomb_melee(old_bab, stats)
        if self.data['Ranged'] != '':
            self.data['Ranged'] = self._get_zomb_ranged(old_bab, stats)
    
        AC_mods, AC_all = self._zombie_AC(stats)
        self.data['AC_Mods'] = AC_mods
        self.data['AC'] = AC_all
        self.data['Init'] = str(math.floor((float(stats[1][1])-10)/2) + 4)
        self.data['Environment'] = 'Any, usually the same as the base creature'
        self.data['SQ']='Staggered'
        self.data['CMB'] = str(math.floor((float(stats[0][1])-10)/2) + math.floor(int(total_hit_dice)*0.75) - int(tt.size_bonus[self.data['Size']]))
        self.data['CMD'] = str(10 + math.floor((float(stats[0][1])-10)/2) + math.floor((float(stats[1][1])-10)/2) + math.floor(int(total_hit_dice)*0.75) - int(tt.size_bonus[self.data['Size']]))
        self.data['Aura'] = ''
        self.data['Spell Like Abilities'] = ''
        self.data['Spells Known'] = ''
        self.data['Spells Prepared'] = ''
        self.data['Spell Domains'] = ''
        self.data['Class'] = ''

    
        return self.data


In [3]:
#url = 'https://www.d20pfsrd.com/bestiary/unique-monsters/cr-11/graveknight/'
#url = 'https://www.d20pfsrd.com/bestiary/monster-listings/oozes/jelly-ochre/'
url = 'https://www.d20pfsrd.com/bestiary/monster-listings/dragons/dragon/metallic-copper/adult-copper-dragon/'
#url = 'https://www.d20pfsrd.com/bestiary/monster-listings/animals/felines/cat/'
#url = 'https://www.d20pfsrd.com/bestiary/npc-s/npcs-cr-8/cultist-human-adept-10/'
#url = 'https://www.d20pfsrd.com/bestiary/monster-listings/humanoids/giants/giant-true/giant-hill/'
#url = 'https://www.d20pfsrd.com/bestiary/monster-listings/animals/crocodilians/crocodile/'
#url = 'https://www.d20pfsrd.com/bestiary/monster-listings/animals/dinosaur/tyrannosaurus/'
#url = 'https://www.d20pfsrd.com/bestiary/monster-listings/animals/birds/owl/'

stat_block = sf.lookup_stats(url)
data_for_mon = stat_block.get_em_all()
zomb_to_be = Apply_Zombie_Template(data_for_mon)
data_for_zomb = zomb_to_be.zombie_temp()
#print(data_for_skel)
sf.print_it_out(data_for_zomb)

Changing fly speed maneuverability to clumsy!
If the base creature flew through magical means, rerun with replace_fly=False
Adult Copper Dragon Zombie	CR 7
XP	3200
 
NE     Large     undead (earth)
Init 4; Senses darkvision 60 ft.

--------------------------------------------------------------
DEFENSE
--------------------------------------------------------------
AC 12, touch 9, flat-footed 12 (, +3 natural, -1 size)
hp 93 (17d8+17)
Fort 5, Ref 5, Will 10 hp 172 (15d12+75)
DR 5/Slashing 
Immune undead traits 

--------------------------------------------------------------
OFFENSE
--------------------------------------------------------------
Speed 40 ft.,  fly 200 ft. (clumsy) ; climb stone
Melee bite +19 (1d8+8), 2 claws +19 (1d6+8), 2 wings +14 (1d6+4), tail +14 (1d6+4), slam +19 (1d8+8) 
Space 10 ft.; 
Reach 5 ft. (10 ft. with bite) 

--------------------------------------------------------------
STATISTICS
--------------------------------------------------------------
Str 27, Dex 1